<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Imports" data-toc-modified-id="Imports-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Imports</a></span></li><li><span><a href="#Open-Serial-Port" data-toc-modified-id="Open-Serial-Port-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Open Serial Port</a></span><ul class="toc-item"><li><span><a href="#Loop-sending-bytes" data-toc-modified-id="Loop-sending-bytes-2.1"><span class="toc-item-num">2.1&nbsp;&nbsp;</span>Loop sending bytes</a></span></li><li><span><a href="#Send-bytes-in-HDLC-in-a-loop" data-toc-modified-id="Send-bytes-in-HDLC-in-a-loop-2.2"><span class="toc-item-num">2.2&nbsp;&nbsp;</span>Send bytes in HDLC in a loop</a></span></li></ul></li><li><span><a href="#Open-wav-file" data-toc-modified-id="Open-wav-file-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>Open wav file</a></span><ul class="toc-item"><li><span><a href="#Just-play-the-song-direcly-in-Python" data-toc-modified-id="Just-play-the-song-direcly-in-Python-3.1"><span class="toc-item-num">3.1&nbsp;&nbsp;</span>Just play the song direcly in Python</a></span></li><li><span><a href="#Send-Data-of-song-via-UART-to-PC-and-play-it-in-Python" data-toc-modified-id="Send-Data-of-song-via-UART-to-PC-and-play-it-in-Python-3.2"><span class="toc-item-num">3.2&nbsp;&nbsp;</span>Send Data of song via UART to PC and play it in Python</a></span></li><li><span><a href="#Send-bytes-of-wav-and-receive-aknowledge" data-toc-modified-id="Send-bytes-of-wav-and-receive-aknowledge-3.3"><span class="toc-item-num">3.3&nbsp;&nbsp;</span>Send bytes of wav and receive aknowledge</a></span></li></ul></li><li><span><a href="#Get-decoded-mp3-data" data-toc-modified-id="Get-decoded-mp3-data-4"><span class="toc-item-num">4&nbsp;&nbsp;</span>Get decoded mp3 data</a></span></li><li><span><a href="#Debug" data-toc-modified-id="Debug-5"><span class="toc-item-num">5&nbsp;&nbsp;</span>Debug</a></span></li></ul></div>

# Imports

In [21]:
import serial
import time
from pathlib import Path

In [22]:
import wave
import pyaudio

In [23]:
import numpy as np

# Open Serial Port

In [26]:
ser = serial.Serial('COM5', 1.5e6, timeout=0, parity=serial.PARITY_NONE) # the baudrate of the uart dongle in verry unprecise. 1.5MHz is matched verry good.

In [25]:
ser.close()

In [11]:
ser.write(bytes([0x03]))#b'U')

1

## Loop sending bytes

In [15]:
while(True):
    for byte in range(0,0x100,1):
        #print(byte)
        ser.write(bytes([byte])) #, 0x01, 0x7F, 0x3F, 0x0F,0xFF]))
        time.sleep(1e-2)

KeyboardInterrupt: 

In [26]:
ser.write(bytes([0x7E]))

1

In [27]:
ser.read_all()

b'\x0f\xfe'

In [ ]:
ser.close()

In [ ]:
while(True):
    time.sleep(1)
    ser.write(bytes([0x00, 0x77, 0x55]))

## Send bytes in HDLC in a loop

In [27]:
def create_hdlc_frame(data):
    if type(data) != bytearray:
        msg = "Unexpected data type. Received {}, but expected 'bytearray'".format(type(data))
    
    frame = b'\x7E' + \
            data.replace(b'\x7D', b'\x7D\x5D') \
                .replace(b'\x7E', b'\x7D\x5E') \
            + b'\x7E'
            
    return frame

In [29]:
data_little_endian = (np.arange(start=0x30, stop=0x40, step=1,dtype=np.int16()))

In [30]:
bytes(data_little_endian)

b'0\x001\x002\x003\x004\x005\x006\x007\x008\x009\x00:\x00;\x00<\x00=\x00>\x00?\x00'

In [31]:
data_big_endian = bytes(data_little_endian.astype('>i2'))
data_big_endian

b'\x000\x001\x002\x003\x004\x005\x006\x007\x008\x009\x00:\x00;\x00<\x00=\x00>\x00?'

In [3]:
#data = bytes(np.arange(start=0x39, stop=0x30, step=-1,dtype=np.int16()))

In [144]:
#data = bytes(np.arange(start=0x39, stop=0x30, step=-1,dtype=np.int16()))

In [32]:
hdlc_frame = create_hdlc_frame(data=data_big_endian)

In [33]:
hdlc_frame

b'~\x000\x001\x002\x003\x004\x005\x006\x007\x008\x009\x00:\x00;\x00<\x00=\x00>\x00?~'

In [ ]:
while(1):
    ser.write(hdlc_frame)
    ser.read_all()

In [ ]:
ser.write(hdlc_frame)
ser.read_all()

In [36]:
data = bytes(np.arange(start=0, stop=0x10, step=1,dtype=np.int16), )

b'\x00\x00\x01\x00\x02\x00\x03\x00\x04\x00\x05\x00\x06\x00\x07\x00\x08\x00\t\x00\n\x00\x0b\x00\x0c\x00\r\x00\x0e\x00\x0f\x00'

In [31]:
data = bytearray(range(0, 0x1FF, 1))
#ser.write(bytes([0x0000, 0x0077, 0x0055]))

ValueError: byte must be in range(0, 256)

In [115]:
a = (b'\x7E')
a

b'~'

# Open wav file

In [7]:
chunk_size = 1024  

In [20]:
wav = wave.open(f=r'..\test\Finjark_Master_003.wav', mode='rb')

## Just play the song direcly in Python

In [13]:
p = pyaudio.PyAudio()  
#open stream  
stream = p.open(format = p.get_format_from_width(wav.getsampwidth()),  
                channels = wav.getnchannels(),  
                rate = wav.getframerate(),  
                output = True)  
#read data  
data = wav.readframes(chunk_size)  

#play stream  
#while data:  
#    stream.write(data)  
#    data = wav.readframes(chunk_size) 

KeyboardInterrupt: 

## Send Data of song via UART to PC and play it in Python

In [85]:
ser.close()
ser = serial.Serial(port='COM5', baudrate=256e3, timeout=5, parity=serial.PARITY_NONE, stopbits=serial.STOPBITS_ONE)

In [114]:
chunk_size = 20
data = None
wav = wave.open(f=str(wav_path / 'Finjark_Master_003.wav'), mode='rb')
p = pyaudio.PyAudio()  
#open stream  
stream = p.open(format = p.get_format_from_width(wav.getsampwidth()),  
                channels = wav.getnchannels(),  
                rate = wav.getframerate(),  
                output = True) 

ser.write(wav.readframes(chunk_size))
#data = ser.read_all()

#read data
#while data:
#    stream.write(data)
#    ser.write(wav.readframes(chunk_size))
#    data = ser.read_all()
    


#play stream  
#while data:  
#    stream.write(data)  
#    data = wav.readframes(chunk_size) 

40

## Send bytes of wav and receive aknowledge

In [46]:
wav_path = Path(r'D:\Sicherungen\Musik_Erwin\Knaat')

In [47]:
def get_transmission_chunk(wav, chunk_size):
    #frame = b'\x7E' + \
    frame = wav.readframes(chunk_size).replace(b'\x7D', b'\x7D\x5D') \
                                      .replace(b'\x7E', b'\x7D\x5E') \
            + b'\x7E'
    return frame

In [49]:
free_mem = 0
response_size = 2

wav = wave.open(f=str(wav_path / 'Finjark_Master_003.wav'), mode='rb')
p = pyaudio.PyAudio()  
#open stream  
stream = p.open(format = p.get_format_from_width(wav.getsampwidth()),  
                channels = wav.getnchannels(),  
                rate = wav.getframerate(),  
                output = True) 


ser.write(b'\x7E')
time.sleep(1e-3)
rec = ser.read(response_size).hex()
if rec == '':
    raise TimeoutError('Timout reached')
free_mem = int(rec, 16)
frame = get_transmission_chunk(wav=wav, chunk_size=free_mem // wav.getsampwidth())

#read data
while frame:
    if free_mem > 0:
        ser.write(frame)
        time.sleep(1e-3)
        rec = ser.read(response_size).hex()
        if rec == '':
            raise TimeoutError('Timout reached')
        free_mem = int(rec, 16)
        frame = get_transmission_chunk(wav=wav, chunk_size=free_mem // wav.getsampwidth())
    else:
        print('FPGA Buffer full!!')
        time.sleep(5e-3)
        ser.write(b'\x7E')
        time.sleep(1e-3)
        rec = ser.read(response_size).hex()
        if rec == '':
            raise TimeoutError('Timout reached')
        free_mem = int(rec, 16)

KeyboardInterrupt: 

In [40]:
for byte in range(0,0x100,1):
    ser.write(bytes([byte]))

In [82]:
wav.getnframes()

9879552

# Get decoded mp3 data

# Debug

In [95]:
a = b'\x7E\x7D'

In [99]:
int(a.hex(), 16)

32381

In [62]:
a.replace(b'\x7D', b'\x7D\x5D').replace(b'\x7E', b'\x7D\x5E').hex()

'7d5e7d5d'

In [55]:
type(wav.readframes(chunk_size))

bytes

In [83]:
ser.timeout

0

In [73]:
(b'\x7E' + wav.readframes(chunk_size).replace(b'\x7D', b'\x7D\x5D').replace(b'\x7E', b'\x7D\x5E') + b'\x7E').hex()

'7e0000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000

In [18]:
ser.close()

In [50]:
wav.